In [2]:
import torch
import h5py
import numpy as np
from pathlib import Path
from openslide import OpenSlide

PATHOLOGY_ROOT = Path('/mnt/hpc/pathology')

WSI_DIR  = PATHOLOGY_ROOT / 'metastasis'
PATCH_DIR = PATHOLOGY_ROOT / 'hipt_preprocessed'
FEATURE_DIR = PATHOLOGY_ROOT / 'hipt_features/primary_vs_metastasis'

CHECKPOINT_PATH = '/home/rens/repos/premium_pathology/hipt/checkpoints/vit256_small_dino.pth'

PATCH_SIZE = 256

center = 'radboud'

slide_path = sorted(list((WSI_DIR / center).iterdir()))[8]
slide = OpenSlide(slide_path)

case_id = slide_path.stem
patches_path = PATCH_DIR / 'metastasis' / center / 'patches' / (case_id + '.h5')
patch_coordinates = h5py.File(patches_path, 'r')['coords']

coords=patch_coordinates[0]


In [3]:
import pytorch_lightning as pl
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from hipt_model_utils import get_vit256


class PatchDataset(Dataset):
    def __init__(self, patch_coordinates, transform):
        self.patch_coordinates = patch_coordinates[:10]
        self.transform = transform

    def __getitem__(self, ix):
        coords = self.patch_coordinates[ix]
        img = self.transform(coords)
        return img

    def __len__(self):
        return self.patch_coordinates.shape[0]


class HIPTFeatureExtractor(pl.LightningModule):
    def __init__(self, slide_path):
        super().__init__()
        self.case_id = slide_path.stem
        self.slide = OpenSlide(slide_path)     

        self.model = get_vit256(CHECKPOINT_PATH)

        self.embeddings = []

    def forward(self, x):
        return self.model(x)
    
    def test_step(self, batch, batch_idx):
        out = self(batch)
        self.embeddings.append(out)

    def setup(self, stage='test'):
        patches_path = PATCH_DIR / 'metastasis' / center / 'patches' / (self.case_id + '.h5')
        patch_coordinates = h5py.File(patches_path, 'r')['coords']

        transform = transforms.Compose([
            transforms.Lambda(lambda coords: np.array(self.slide.read_region(coords,1,(PATCH_SIZE,PATCH_SIZE)))[:,:,:-1]),
            transforms.ToTensor(),
            transforms.Normalize(mean=0.5, std=0.5)
        ])

        self.dataset = PatchDataset(patch_coordinates, transform) 

    def test_dataloader(self):
        return DataLoader(self.dataset, batch_size=1, shuffle=False, num_workers=24)

    
fe = HIPTFeatureExtractor(slide_path)
trainer = pl.Trainer(accelerator='gpu')

trainer.test(fe)



GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: /home/rens/repos/premium_pathology/hipt/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Take key teacher in provided checkpoint dict
Pretrained weights found at /home/rens/repos/premium_pathology/hipt/checkpoints/vit256_small_dino.pth and loaded with msg: _IncompatibleKeys(missing_keys=[], unexpected_keys=['head.mlp.0.weight', 'head.mlp.0.bias', 'head.mlp.2.weight', 'head.mlp.2.bias', 'head.mlp.4.weight', 'head.mlp.4.bias', 'head.last_layer.weight_g', 'head.last_layer.weight_v'])


Testing: 0it [00:00, ?it/s]

: 

: 

In [21]:
x.to('cuda:0')

AssertionError: Torch not compiled with CUDA enabled

In [7]:
fe.setup()

fe.dataset[0].shape

torch.Size([3, 256, 256])

In [8]:
dl = fe.test_dataloader()
x = next(iter(dl))

In [9]:
out = model(x)

In [11]:
out.shape

torch.Size([2, 384])

In [61]:
import matplotlib.pyplot as plt

for x in dl:
    break

In [62]:
x.numpy()[0].shape

(3, 256, 256)

In [3]:
import h5py

patches = Path('/mnt/hpc/pathology/clam_preprocessed/metastasis/zuyderland/patches/PREM_ZU_038 - T20-020312-I-S-1 - 2021-08-12 15.16.35.h5')

p = h5py.File(patches, 'r')

In [13]:
p['coords'][0]

array([ 3520, 81664])

In [14]:
q = h5py.File(
    '/mnt/hpc/pathology/clam_features/primary_vs_metastasis/h5_files/PREM_ZU_038 - T20-020312-I-S-1 - 2021-08-12 15.16.35.h5'
    , 'r')

In [15]:
q['coords'][0]

array([ 3520, 81664])

In [17]:
x.shape

torch.Size([58808, 1024])